In [1]:
from sklearn import cluster, datasets, mixture
import numpy as np
import pandas as pd

import MBMM
from MBMM import MBMM
import FBBMM
from FBBMM import FBBMM

import random
from sklearn.cluster import KMeans, MeanShift, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn import metrics
import warnings

from keras.datasets import mnist

## Calculate accuracy reference from: https://github.com/sharmaroshan/MNIST-Using-K-means  
def infer_cluster_labels(pred_labels, actual_labels):
    """
    Associates most probable label with each cluster in KMeans model
    returns: dictionary of clusters assigned to each label
    """
    
    inferred_labels = {}
    n_clusters = len(set(actual_labels))
    #n_clusters = len(np.unique(pred_labels))
    for i in range(n_clusters):

        # find index of points in cluster
        labels = []
        index = np.where(pred_labels == i)

        # append actual labels for each point in cluster
        labels.append(actual_labels[index])

        # determine most common label
        if len(labels[0]) == 1:
            counts = np.bincount(labels[0])
        else:
            counts = np.bincount(np.squeeze(labels))

        # assign the cluster to a value in the inferred_labels dictionary
        if len(counts) > 0:        
            if np.argmax(counts) in inferred_labels:
                # append the new number to the existing array at this slot
                inferred_labels[np.argmax(counts)].append(i)
            else:
                # create a new array in this slot
                inferred_labels[np.argmax(counts)] = [i]    

    return inferred_labels  
    
    
def infer_data_labels(X_labels, cluster_labels):
    """
    Determines label for each array, depending on the cluster it has been assigned to.
    returns: predicted labels for each array
    """
    predicted_labels = np.array([-1 for i in range(len(X_labels))])
    
    for i, cluster in enumerate(X_labels):
        for key, value in cluster_labels.items():
            if cluster in value:
                predicted_labels[i] = key
                
    return predicted_labels

def load_data():
    data = pd.read_csv('mnist_2d.csv').to_numpy() #70000,2
    
    (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
    lower, upper = 0.01, 0.99
    data = lower + (data - np.min(data))*(upper-lower)/(np.max(data)-np.min(data))
    target = np.concatenate((Y_train, Y_test), axis = 0)
    return data, target

def data_to_target(data, target):
    data_dict = {}
    for i in range(len(data)):
        if target[i] not in data_dict.keys():
            data_dict[target[i]] = []
            data_dict[target[i]].append(data[i])
        else:
            data_dict[target[i]].append(data[i])
            
    return data_dict


def smaple_number(num_list, data_dict):
    data = []
    target = []
    total_train = 0
    for n in num_list:  
        data = np.append(data, data_dict[n])        
        target = np.append(target, [n for _ in range(len(data_dict[n]))])       
        total_train += len(data_dict[n]) #累積的資料數 
    data = np.reshape(np.array(data), (total_train, 2))
    target = np.reshape(np.array(target), (total_train)).astype(int)
    
    return data, target
    

def initial_param(data, target):
    # ============
    # Initial parameters
    # ============
    MBMM_param38 = np.array([[1.09717833, 8.04592969, 8.0686254 ],
                             [2.84798971, 2.20653172, 1.52946912]])
    
    FBBMM_param38 = np.array([[3.14190044, 6.35378162, 4.94796333, 3.971114  ],
                         [8.28131719, 8.89802408, 6.70568243, 3.18620078]])
    
    MBMM_param17 = np.array([[4.00195568, 9.09896669, 4.21570771],
                                 [2.2397298,  2.91100834, 3.6114422 ]])
    

    FBBMM_param17 = np.array([[9.64563123, 6.51637147, 7.77768709, 3.65455795],
                                 [2.15407161, 3.84410293, 4.04455582, 8.6491259 ]])
    
    data_dict = data_to_target(data, target)
    
    data_38, target_38 = smaple_number([3,8], data_dict)
    data_17, target_17 = smaple_number([1,7], data_dict)
     
    param = [(data_38, {'n_clusters': 2, 'quantile': .22, 'eps': .16, 'min_samples': 4140,'linkage': "ward", 
                     'affinity': "euclidean", 'MBMM_param': MBMM_param38, 'FBBMM_param':FBBMM_param38}),
            (data_17, {'n_clusters': 2, 'quantile': .35, 'eps': .11, 'min_samples': 2300, 'linkage': "ward", 
                       'affinity': "euclidean", 'MBMM_param': MBMM_param17, 'FBBMM_param':FBBMM_param17})]
    target = [target_38, target_17]
    
    return param, target


if __name__ == "__main__":
    mnist_data, mnist_target = load_data()
               
    parameters, target = initial_param(mnist_data, mnist_target)

    for i_dataset, (dataset, params) in enumerate(parameters):  

        kmeans = cluster.KMeans(n_clusters=params['n_clusters'])
               
        bandwidth = cluster.estimate_bandwidth(dataset, quantile=params['quantile'])
        ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)

        dbscan = cluster.DBSCAN(eps=params['eps'], min_samples=params['min_samples'])

        aggolmarative = cluster.AgglomerativeClustering(
            linkage=params['linkage'],
            affinity=params['affinity'],
            n_clusters=params['n_clusters'],
        )

        gmm = mixture.GaussianMixture(n_components=params['n_clusters'], covariance_type='full')

        mbmm = MBMM(n_components=params['n_clusters'], n_runs=100, param=params['MBMM_param'], tol=1e-3)
        
        fbbmm = FBBMM(n_components=params['n_clusters'], n_runs=20, param=params['FBBMM_param'], tol=1e-2)   

        clustering_algorithms = (
            ('K-means', kmeans),
            ("MeanShift", ms),
            ("DBSCAN", dbscan),
            ("AgglomerativeClustering", aggolmarative),
            ('GMM', gmm),
            ('MBMM', mbmm),
            ('FBBMM', fbbmm))
        
        #print result
        if i_dataset == 0:     
            print('number3 and 8:')
        if i_dataset == 1:
             print('number1 and 7:')
                
        for name, algorithm in clustering_algorithms:
            algorithm.fit(dataset)

            if hasattr(algorithm, 'labels_'):
                train_predict_y = algorithm.labels_.astype(int)
            else:
                train_predict_y = algorithm.predict(dataset)        
           
            cluster_labels = infer_cluster_labels(train_predict_y, target[i_dataset])
            train_predicted_labels = infer_data_labels(train_predict_y, cluster_labels)       
            acc = np.round(np.count_nonzero(target[i_dataset] == train_predicted_labels)/len(target[i_dataset]), 3)

            ari_value = np.round(metrics.adjusted_rand_score(target[i_dataset], train_predict_y), 3)

            ami_value = np.round(metrics.adjusted_mutual_info_score(target[i_dataset], train_predict_y), 3)
                   
            print(name, {'Accuracy':acc, 'ARI':ari_value, "AMI":ami_value})


number3 and 8:
K-means {'Accuracy': 0.839, 'ARI': 0.459, 'AMI': 0.367}
MeanShift {'Accuracy': 0.823, 'ARI': 0.484, 'AMI': 0.374}
DBSCAN {'Accuracy': 0.347, 'ARI': 0.44, 'AMI': 0.387}
AgglomerativeClustering {'Accuracy': 0.835, 'ARI': 0.45, 'AMI': 0.355}
GMM {'Accuracy': 0.831, 'ARI': 0.439, 'AMI': 0.347}
MBMM {'Accuracy': 0.832, 'ARI': 0.44, 'AMI': 0.371}


/home/dart/peng/FBBMM.py:98: RuntimeWarning: divide by zero encountered in log
  log_prob[n,c] = np.log(val)+self.log_gamma_function(np.sum(para))-np.sum(self.log_gamma_function(para))
/home/dart/peng/FBBMM.py:96: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  val = integrate.quad(f, lower_bound, upper_bound, args=(x[0],x[1]))[0]
/home/dart/peng/FBBMM.py:96: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  val = integrate.quad(f, lower_bound, upper_bound, args=(x[0],x[1]))[0]
/home/dart/peng/FBBMM.py:96: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local

FBBMM {'Accuracy': 0.829, 'ARI': 0.433, 'AMI': 0.349}
number1 and 7:
K-means {'Accuracy': 0.971, 'ARI': 0.889, 'AMI': 0.813}
MeanShift {'Accuracy': 0.975, 'ARI': 0.903, 'AMI': 0.832}
DBSCAN {'Accuracy': 0.944, 'ARI': 0.857, 'AMI': 0.761}
AgglomerativeClustering {'Accuracy': 0.973, 'ARI': 0.897, 'AMI': 0.823}
GMM {'Accuracy': 0.97, 'ARI': 0.883, 'AMI': 0.806}
MBMM {'Accuracy': 0.93, 'ARI': 0.738, 'AMI': 0.633}


/home/dart/peng/FBBMM.py:96: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  val = integrate.quad(f, lower_bound, upper_bound, args=(x[0],x[1]))[0]
/home/dart/peng/FBBMM.py:96: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  val = integrate.quad(f, lower_bound, upper_bound, args=(x[0],x[1]))[0]
/home/dart/peng/FBBMM.py:96: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-pu

FBBMM {'Accuracy': 0.976, 'ARI': 0.907, 'AMI': 0.841}
